# Exploring GLAD ARD data.

In [2]:
import math
import json
from datetime import datetime
from urllib.parse import urlsplit
import rasterio
from rasterio import windows, transform
from rasterio.vrt import WarpedVRT
from pyproj import CRS, Transformer

import boto3
import mercantile
import geopandas as gpd
import pystac
import requests
import numpy as np
from requests.auth import HTTPBasicAuth
import stac_geoparquet
from shapely.geometry import Polygon
from lonboard import viz, Map, PolygonLayer
from supermercado import burntiles

In [3]:
# Download from https://glad.umd.edu/ard/home
index_fpath = "/Users/jeffreyalbrecht/Downloads/Global_ARD_tiles/glad_ard_tiles.shp"
df = gpd.read_file(index_fpath)

In [4]:
viz(df)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [5]:
z7_tiles = burntiles.burn(json.loads(df.to_json())['features'], 7)
z7_feats = [mercantile.feature(tile, props={"quadkey": mercantile.quadkey(mercantile.Tile(*tile))}) for tile in z7_tiles]
z7_df = gpd.GeoDataFrame.from_features(z7_feats)

/Users/jeffreyalbrecht/miniconda3/envs/cog-layers/lib/python3.11/site-packages/mercantile/__init__.py:77: FutureWarning: Mercantile 2.0 will require tile x and y to be within the range (0, 2 ** zoom)
  warnings.warn(


In [6]:
# A GeoDataFrame with Polygon geometries
glad_layer = PolygonLayer.from_geopandas(
    df,
    get_fill_color=(189,189,189, 255), get_line_color=[0, 0, 0, 200], line_width_min_pixels=0.25, opacity=0.5
)
z7_layer = PolygonLayer.from_geopandas(
    z7_df,
    get_line_color=[222,45,38, 200],
    line_width_min_pixels=3,
    line_width_max_pixels=2,
    opacity=0.5,
    filled=False
)

m = Map([glad_layer, z7_layer])
m

/Users/jeffreyalbrecht/miniconda3/envs/cog-layers/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(custom_attribution='', layers=(PolygonLayer(get_fill_color=(189, 189, 189, 255), get_line_color=[0, 0, 0, …

# Create a catalog

In [7]:
with open("glad_ard.jsonl", "w") as outf:
    for row in df.itertuples():
        feat = {
            "id": row.TILE,
            "bbox": row.geometry.bounds,
            "datetime": datetime(2020, 1, 1),
            "geometry": row.geometry.__geo_interface__,
            "properties": {"foo": "bar"},
            "assets": {"img": pystac.Asset(href=f"s3://glad.landsat.ard/data/tiles/{row.TILE.split('_')[-1]}/{row.TILE}/{{interval}}.tif")}
            # "assets": {"img": pystac.Asset(href=f"https://glad.umd.edu/dataset/glad_ard2/{row.TILE.split('_')[-1]}/{row.TILE}/<interval>.tif")},
        }
        item = pystac.Item(**feat)
        json.dump(item.to_dict(), outf, separators=(",", ":"))
        outf.write("\n")

In [8]:
stac_geoparquet.arrow.parse_stac_ndjson_to_parquet("glad_ard.jsonl", "glad_ard.parquet")

In [9]:
viz(gpd.read_parquet("glad_ard.parquet"))

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

# Create a single COG for a given interval

In [26]:
df = gpd.read_parquet("glad_ard.parquet")
quadkey = "0231102"
tile = mercantile.quadkey_to_tile("0231102")
tile_geom = Polygon.from_bounds(*mercantile.bounds(*tile))
glad_tiles = df.loc[df.intersects(tile_geom)==True]


# Download the files.
s3_client = boto3.client('s3', region_name='us-east-1')
for idx, glad_tile in enumerate(glad_tiles.itertuples()):
    uri = glad_tile.assets['img']['href'].format(interval=994)
    print(uri)
    splits = urlsplit(uri)
    bucket = splits.netloc
    key = splits.path[1:]

    # s3_client.download_file(Filename=f"data/{idx}.tif",Bucket=bucket, Key=key)

s3://glad.landsat.ard/data/tiles/38N/101W_38N/994.tif
s3://glad.landsat.ard/data/tiles/38N/100W_38N/994.tif
s3://glad.landsat.ard/data/tiles/38N/099W_38N/994.tif
s3://glad.landsat.ard/data/tiles/38N/098W_38N/994.tif
s3://glad.landsat.ard/data/tiles/37N/101W_37N/994.tif
s3://glad.landsat.ard/data/tiles/37N/100W_37N/994.tif
s3://glad.landsat.ard/data/tiles/37N/099W_37N/994.tif
s3://glad.landsat.ard/data/tiles/37N/098W_37N/994.tif
s3://glad.landsat.ard/data/tiles/36N/101W_36N/994.tif
s3://glad.landsat.ard/data/tiles/36N/100W_36N/994.tif
s3://glad.landsat.ard/data/tiles/36N/099W_36N/994.tif
s3://glad.landsat.ard/data/tiles/36N/098W_36N/994.tif


In [27]:
%%bash
gdalbuildvrt mosaic.vrt data/*

0...10...20...30...40...50...60...70...80...90...100 - done.


In [28]:
def calculate_resolution(zoom, lat = 0):
    earth_circumference = 40075016.686  # in meters
    resolution = (earth_circumference * math.cos(math.radians(lat))) / (2**(zoom + 8))
    return resolution

In [30]:
from rasterio.enums import Resampling
from rio_tiler.utils import linear_rescale, _weighted_quantiles


OVERVIEW_COUNT = 5

with rasterio.open("mosaic.vrt") as ds:
    with WarpedVRT(ds, crs="EPSG:3857") as vrt:
        # The image should have bounds of a single Z7 tile.
        tile_bounds = mercantile.xy_bounds(tile)
        parent_tile_window = windows.from_bounds(*tile_bounds, vrt.transform)
        # The native resolution of the image should be that of a Z12 tile (~38m at equator).
        native_res = calculate_resolution(tile.z + OVERVIEW_COUNT)
        expected_width = expected_height = 2 ** (12 - 7) * 256


        # TODO: mask with band8
        arr = vrt.read(
            [3, 2, 1],
            window=parent_tile_window,
            resampling=Resampling.bilinear,
            out_shape=(3, expected_height, expected_width)
        )

        # 16bit -> 8bit stretch (via rio-tiler.utils)
        for i in range(arr.shape[0]):
            p2 = _weighted_quantiles(arr[i].flatten(), arr[i].flatten(), 0.02)
            p98 = _weighted_quantiles(arr[i].flatten(), arr[i].flatten(), 0.90)
            arr[i] = linear_rescale(arr[i], (p2, p98)).astype('uint8')
        arr = arr.astype('uint8')
            
        out_profile = {
            "driver": "COG",
            "width": expected_width,
            "height": expected_height,
            "dtype": arr.dtype,
            "count": arr.shape[0],
            "crs": vrt.crs,
            "transform": transform.from_bounds(*tile_bounds, expected_width, expected_height),
            "blocksize": 256,
            "compress": "JPEG",
            "overview_count": OVERVIEW_COUNT,
        }
        print(out_profile)
        with rasterio.open(f"{quadkey}.tif", "w", **out_profile) as dst:
            dst.write(arr)

{'driver': 'COG', 'width': 8192, 'height': 8192, 'dtype': dtype('uint8'), 'count': 3, 'crs': CRS.from_epsg(3857), 'transform': Affine(38.21851414258822, 0.0, -11271098.442818949,
       0.0, -38.218514142588106, 4696291.017841229), 'blocksize': 256, 'compress': 'JPEG', 'overview_count': 5}


In [31]:
%%bash
gdalinfo test_webcog.tif

Driver: GTiff/GeoTIFF
Files: test_webcog.tif
Size is 8192, 8192
Coordinate System is:
4 / Pseudo-Mercator",
    BASEGEOGCRS["WGS 84",
e",     ENSEMBLE["World Geodetic System 1984 ensembl
            MEMBER["World Geodetic System 1984 (Transit)"],
ystem 1984 (G730)"],World Geodetic S
            MEMBER["World Geodetic System 1984 (G873)"],
rld Geodetic System 1984 (G1150)"],
            MEMBER["World Geodetic System 1984 (G1674)"],
      MEMBER["World Geodetic System 1984 (G1762)"],
G2139)"],   MEMBER["World Geodetic System 1984 (
            ELLIPSOID["WGS 84",6378137,298.257223563,
]],             LENGTHUNIT["metre",1
            ENSEMBLEACCURACY[2.0]],
        PRIMEM["Greenwich",0,
,0.0174532925199433]],"degree"
        ID["EPSG",4326]],
or",CONVERSION["Popular Visualisation Pseudo-Mercat
        METHOD["Popular Visualisation Pseudo Mercator",
            ID["EPSG",1024]],
ARAMETER["Latitude of natural origin",0,
            ANGLEUNIT["degree",0.0174532925199433],
     ID["EPSG",8801